# MicroPython Code

ADCs map analog input voltage to a digital code. The figure below shows the relationship for several gain settings. Since our signal is centered around $V_\textrm{ref}\approx 1.65V$ we use the 11dB attenuation setting. 

![](figures/esp32_adc_vin.png)

Below is the code to setup and read the ADC, adapted from the example in the [MicroPython documentation](https://docs.micropython.org/en/latest/esp32/quickref.html#adc-analog-to-digital-conversion).


In [1]:
%discover
%connect huzzah32

from machine import ADC, Pin
import time

# configure ADC3 (output of the INA126)
out = ADC(Pin(39))
out.atten(ADC.ATTN_11DB)

# configure ADC6 (Vref)
ref = ADC(Pin(34))
ref.atten(ADC.ATTN_11DB)

# read the ADCs in a loop and display the result
# _ just means that we don't care for the loop counter
for _ in range(10):
    vout = out.read()
    vref = ref.read()
    print("out = {:4}  ref = {:4}   delta = {:4}".format(vout, vref, vout-vref))
    time.sleep(1)

huzzah32  serial:///dev/ttyUSB0  
Connected to huzzah32 @ serial:///dev/ttyUSB0
out = 1789  ref = 1729   delta =   60
out = 1776  ref = 1719   delta =   57
out = 1776  ref = 1717   delta =   59
out = 1776  ref = 1724   delta =   52
out = 1781  ref = 1709   delta =   72
out = 1776  ref = 1713   delta =   63
out = 1783  ref = 1727   delta =   56
out = 1778  ref = 1723   delta =   55
out = 1808  ref = 1721   delta =   87
out = 1781  ref = 1715   delta =   66


Playing around with the scale, you can see that the output (`delta`) changes when you put a weight (e.g. your finger) on the scale. But even with no weight applied, the `delta` is not zero. This error is called "offset" and comes from inacurracies in the load cell, the INA216, and the ADC.

Further, values reported by the ADC change even for constant weight. This "noise" is the result of electrical interference and the ADC.

Let's try averaging a few samples to see if we can reduce the noise:

In [1]:
for N in [1, 10, 100]:
    for _ in range(5):
        vout = 0
        vref = 0
        for _ in range(N):
            vout += out.read()
            vref += ref.read()
        vout /= N
        vref /= N
        print("N = {:3}  out = {:4.0f}  ref = {:4.0f}   delta = {:4.0f}".format(
            N, vout, vref, vout-vref))
        time.sleep(1)
    print()

N =   1  out = 1807  ref = 1725   delta =   82
N =   1  out = 1782  ref = 1722   delta =   60
N =   1  out = 1792  ref = 1725   delta =   67
N =   1  out = 1776  ref = 1728   delta =   48
N =   1  out = 1776  ref = 1731   delta =   45

N =  10  out = 1779  ref = 1719   delta =   60
N =  10  out = 1781  ref = 1724   delta =   57
N =  10  out = 1780  ref = 1727   delta =   53
N =  10  out = 1778  ref = 1723   delta =   55
N =  10  out = 1778  ref = 1728   delta =   51

N = 100  out = 1780  ref = 1724   delta =   56
N = 100  out = 1780  ref = 1723   delta =   57
N = 100  out = 1779  ref = 1725   delta =   54
N = 100  out = 1778  ref = 1723   delta =   54
N = 100  out = 1778  ref = 1722   delta =   55



In these tests I did not apply any force to the scale. 

Averaging definitely helps. In my trials it reduced the noise (variations of `delta`) from almost 40 without averaging (N=1) to less than 5 (N=100), an almost ten-fold improvement!

Let's update the code again, this time first measuring the offset and then subtracting it from subsequent measurements. We also create a function for reading the ADC and averaging its outputs. In `read_adc` we average the difference, a small optimization to keep the sum smaller, even for large N. 

In [1]:
def read_adc(out, ref, N=100):
    sum = 0
    for _ in range(N):
        sum += out.read() - ref.read()
    return sum/N

# measure the offset
offset = read_adc(out, ref)

# weigh ...
for _ in range(20):
    weight = read_adc(out, ref) - offset
    print("weight = {:4.0f}".format(weight))
    time.sleep(1)

weight =    0
weight =   -0
weight =    2
weight =  267
weight =  361
weight =   -0
weight =    0
weight =    3
weight = -435
weight = -308
weight =    0
weight =   -2
weight =    2
weight =    2
weight =   -0
weight =   -0
weight =    0
weight =    1
weight =    0
weight =   -1


Not perfect but quite usable.

Let's now calibrate the scale so it's output is in grams. For this we need a reference with known weight. If you do not have calibrated weights just get something with a weight close to the full scale of your load cell, get another scale to determine its weight, and then put it on your scale.

In [1]:
offset = read_adc(out, ref)

for _ in range(5):
    print("weight = {:4.0f}".format(read_adc(out, ref) - offset))
    time.sleep(2)

weight =    0
weight =    1
weight =  854
weight =  841
weight =  839


My reference weighs 500grams. The output of the scale is about 842 (averaged), so let's redo the test with the output scaled by 500/842.

In [1]:
offset = read_adc(out, ref)

for _ in range(5):
    weight = read_adc(out, ref) - offset
    weight_scaled = weight * 500 / 843
    print("weight = {:4.0f} grams".format(weight_scaled))
    time.sleep(2)

weight =    0 grams
weight =    6 grams
weight = -499 grams
weight = -497 grams
weight = -498 grams


Ups, I forgot to remove the weight before I started the test. Now it comes out negative: I removed 500grams.

Actually that's quite useful, we can use this feature to implement a "tare" function: a button that, when pressed, zeroes the output. Very useful when you bake a cake to "null" the weight of the container you put the contents in.